In [1]:
from utils import *
from CS_feature_extractor import *
from CS_based_early_stopping import *

/home/guangya/SC_conf/src/utils.py:375: SyntaxWarning: invalid escape sequence '\%'
  string = string.replace("\%", "")
[nltk_data] Downloading package punkt to /home/guangya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guangya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# How to run the code to get Acc and # of API calls

In [2]:
DATA_DIR = "../data/Evaluation_CoTs/"
input_file_path = os.path.join(DATA_DIR, 'llama3.csv')
df_llama = pd.read_csv(input_file_path).reset_index(drop=True)

In [3]:
df_test = df_llama.sample(50).reset_index(drop=True) # get partial data for test

In [4]:
df_test  # Please put your question and CoT in the given format

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39,Difficulty,Model
0,MathQA_dev,Math,if a function f is defined by f ( x ) = - 1 / ...,d,Let's solve this problem step by step.\n\nWe k...,E,"[(0, 1)]","To solve for b, we can start by using the give...",E,"[(0, 1)]",...,E,"[(0, 1)]",Let's break down the problem step by step.\n\n...,E,"[(0, 1)]",Let's break down the problem step by step.\n\n...,A,"[(0, 1)]",easy,llama3
1,MathQA_dev,Math,what will be in unit ' s place digit of 3 to t...,e,"To solve this problem, we need to find the uni...",E,"[(0, 1)]",Let's tackle this problem step by step.\n\nWe'...,E,"[(0, 1)]",...,E,"[(0, 1)]",To find the unit's place digit of 3 to the pow...,E,"[(0, 1)]",Let's solve this step by step.\n\nThe problem ...,E,"[(0, 1)]",easy,llama3
2,BigBench_easy,movie_recommendation,Find a movie similar to Terminator 2 Judgment ...,C,What an intriguing combination of movies! To f...,D,"[(0, 1)]",To find a movie similar to Terminator 2 Judgme...,E,"[(0, 1)]",...,D,"[(0, 1)]",To find a movie similar to Terminator 2 Judgme...,E,"[(0, 1)]",To find a movie similar to Terminator 2 Judgme...,D,"[(0, 1)]",easy,llama3
3,BigBench_easy,movie_recommendation,Find a movie similar to The Shawshank Redempti...,C,What a great set of movies to compare with! To...,A,"[(0, 1)]",To find a movie similar to The Shawshank Redem...,E,"[(0, 1)]",...,E,"[(0, 1)]",To find a movie similar to The Shawshank Redem...,E,"[(0, 1)]",What a great question!\n\nTo find a movie simi...,A,"[(0, 1)]",easy,llama3
4,GSM8K_test,Math,Mike plays ping pong for 40 minutes. In the f...,9,Let's break this problem down step by step.\n\...,9,"[(0, 1)]","To solve this problem, we need to calculate th...",9,"[(0, 1)]",...,9,"[(0, 1)]",Let's break this problem down step by step.\n\...,40,"[(0, 1)]",I'd be happy to help you with that!\n\nLet's b...,40,"[(0, 1)]",easy,llama3
5,BigBench_easy,movie_recommendation,"Find a movie similar to The Matrix, Seven, The...",A,What an intriguing set of movies! Let's break ...,E,"[(0, 1)]","To find a movie similar to The Matrix, Seven, ...",C,"[(0, 1)]",...,C,"[(0, 1)]",What an intriguing set of movies! Let's break ...,C,"[(0, 1)]","To find a movie similar to The Matrix, Seven, ...",C,"[(0, 1)]",easy,llama3
6,BigBench_easy,movie_recommendation,Find a movie similar to Terminator 2 Judgment ...,D,To find a movie similar to Terminator 2 Judgme...,D,"[(0, 1)]",To find a movie similar to Terminator 2 Judgme...,E,"[(0, 1)]",...,E,"[(0, 1)]",To find a movie similar to Terminator 2 Judgme...,E,"[(0, 1)]",To find a movie similar to Terminator 2 Judgme...,E,"[(0, 1)]",easy,llama3
7,MathQA_challenge_test,Challenging Math,find the value of ( 875 233 / 899 ) × 899 The ...,c,To evaluate the expression `(875 + 233) / 899`...,E,"[(0, 1)]","To find the value of (875 ÷ 233) × 899, we'll ...",C,"[(0, 1)]",...,E,"[(0, 1)]",I'd be happy to help you with that!\n\nTo eval...,E,"[(0, 1)]","To find the value of `(875 ÷ 233) × 899`, we n...",E,"[(0, 1)]",hard,llama3
8,BigBench_hard,disambiguation_qa,"In the following sentences, explain the antece...",B,"To determine the antecedent of the pronoun ""he...",C,"[(0, 1)]","To determine the antecedent of the pronoun ""he...",A,"[(0, 1)]",...,A,"[(0, 1)]","To identify the antecedent of the pronoun ""he""...",E,"[(0, 1)]","The antecedent of the pronoun ""he"" in this sen...",E,"[(0, 1)]",hard,llama3
9,BigBench_easy,movie_recommendation,"Find a movie similar to Seven, Star Wars Episo...",C,"To find a movie similar to Seven, Star Wars Ep...",D,"[(0, 1)]",What a fascinating combination of movies! To f...,D,"[(0, 1)]",...,D,"[(0, 1)]","To find a movie similar to Seven, Star Wars Ep...",D,"[(0, 1)]","To find a movie similar to Seven, Star Wars Ep...",D,"[(0, 1)]",easy,llama3


In [5]:
feature_li = ['LEN', 'QUA_IM', 'DIF_IV', 'SIM_COT_BIGRAM', 'SIM_COT_AGG', 'SIM_AC_BIGRAM', 'SIM_AC_AGG', 'SIM_INPUT', 'STEP_COUNT', 'AVG_STEP_LENGTH', 'MATH_TERM_DENSITY', 'IMPERATIVE_DENSITY', 'STEP_COHERENCE'] 
# This can be different; please see extract features for more details
data = extract_feature(df_test,feature_li)

jaccard with bigram time cost: 2.957132577896118s
jaccard with aggregation time cost: 28.815580368041992s


100%|██████████| 50/50 [00:03<00:00, 14.32it/s]


In [6]:
pd.DataFrame(data).head(5) # data is saved in json format

,id,Name,Model,correct answer,CoT answers,Correctness,MATH_TERM_DENSITY,STEP_COHERENCE,SIM_AC_AGG,SIM_COT_AGG,SIM_INPUT,IMPERATIVE_DENSITY,SIM_COT_BIGRAM,AVG_STEP_LENGTH,LEN,DIF_IV,QUA_IM,STEP_COUNT,SIM_AC_BIGRAM
0,0,MathQA_dev,llama3,d,"[E, E, E, E, E, A, E, A, E, D, E, E, E, E, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.004878048780487805, 0.0, 0.00309597523...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, ...","[0.0, 0.4563106796116505, 0.42647058823529416,...","[0.2178217821782178, 0.2168674698795181, 0.230...","[0.008130081300813009, 0.004878048780487805, 0...","[0.0, 0.4563106796116505, 0.44859813084112155,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ..."
1,1,MathQA_dev,llama3,e,"[E, E, D, B, E, E, C, E, E, E, E, E, E, E, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.04745762711864407, 0.026246719160104987, 0....","[0.27710437710437713, 0, 0, 0, 0.3196721311475...","[0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.0, 0.3076923076923077, 0.2817679558011049, ...","[0.1652892561983471, 0.22999999999999998, 0.19...","[0.030508474576271188, 0.015748031496062992, 0...","[0.0, 0.3076923076923077, 0.30000000000000004,...","[67.0, 0, 0, 0, 186.0, 30.666666666666668, 54....","[0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 0, 0, 0, 2, 3, 2, 0, 0, 3, 2, 0, 1, 2, 3, ...","[0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, ..."
2,2,BigBench_easy,llama3,C,"[D, E, D, A, E, D, D, E, A, E, A, E, D, D, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.0, 0.4104938271604939, 0.3877005347593583, ...","[0.1576576576576577, 0.15254237288135597, 0.18...","[0.0015408320493066256, 0.00338409475465313, 0...","[0.0, 0.4104938271604939, 0.3682539682539683, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,3,BigBench_easy,llama3,C,"[A, E, E, A, A, A, E, E, D, E, A, A, E, E, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2267614798289...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0.0, 0.32770270270270274, 0.3370786516853933,...","[0.16267942583732053, 0.18279569892473113, 0.1...","[0.0022123893805309734, 0.00631578947368421, 0...","[0.0, 0.32770270270270274, 0.35185185185185186...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 140.66666666666...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ..."
4,4,GSM8K_test,llama3,9,"[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 40.0, 9.0,...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, ...","[0.005235602094240838, 0.026041666666666668, 0...","[0, 0, 0, 0, 0, 0, 0.3406447324692589, 0, 0, 0...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, ...","[0.0, 0.48913043478260865, 0.6261682242990654,...","[0.34285714285714286, 0.22784810126582278, 0.2...","[0.015706806282722512, 0.036458333333333336, 0...","[0.0, 0.48913043478260865, 0.5353535353535354,...","[0, 0, 0, 0, 0, 0, 77.33333333333333, 0, 0, 0,...","[1, 0, 1, 0, 1, 1, 0

In [7]:
df_llama_processed = pd.DataFrame(data)

In [21]:
df_llama_processed = calculate_SC_correctness(df_llama_processed)

# Calculate Early Stopping Correctness with a specific window size
window_size = 5  # Define your window size
df_llama_processed = calculate_ES_correctness(df_llama_processed, window_size)

# Calculate Adaptive Consensus Correctness
df_llama_processed = calculate_ASC_correctness(df_llama_processed)

In [23]:
df_llama_processed.head()#

,id,Name,Model,correct answer,CoT answers,Correctness,MATH_TERM_DENSITY,STEP_COHERENCE,SIM_AC_AGG,SIM_COT_AGG,...,DIF_IV,QUA_IM,STEP_COUNT,SIM_AC_BIGRAM,Stratify,SC_correctness,ES_correctness,ES_steps,asc_correctness,asc_steps
0,0,MathQA_dev,llama3,d,"[E, E, E, E, E, A, E, A, E, D, E, E, E, E, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.004878048780487805, 0.0, 0.00309597523...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, ...","[0.5, 0.4563106796116505, 0.42647058823529416,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...",llama3_MathQA_dev,0,0,5,0,4
1,1,MathQA_dev,llama3,e,"[E, E, D, B, E, E, C, E, E, E, E, E, E, E, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.04745762711864407, 0.026246719160104987, 0....","[0.27710437710437713, 0, 0, 0, 0.3196721311475...","[0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.5, 0.3076923076923077, 0.2817679558011049, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 0, 0, 0, 2, 3, 2, 0, 0, 3, 2, 0, 1, 2, 3, ...","[0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...",llama3_MathQA_dev,1,1,12,1,9
2,2,BigBench_easy,llama3,C,"[D, E, D, A, E, D, D, E, A, E, A, E, D, D, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.5, 0.4104938271604939, 0.3877005347593583, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...",llama3_BigBench_easy,0,0,40,0,40
3,3,BigBench_easy,llama3,C,"[A, E, E, A, A, A, E, E, D, E, A, A, E, E, A, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2267614798289...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0.5, 0.32770270270270274, 0.3370786516853933,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ...","[0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, ...",llama3_BigBench_easy,0,0,40,0,40
4,4,GSM8K_test,llama3,9,"[9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0, 40.0, 9.0,...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, ...","[0.005235602094240838, 0.026041666666666668, 0...","[0, 0, 0, 0, 0, 0, 0.3406447324692589, 0, 0, 0...","[0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, ...","[0.5, 0.48913043478260865, 0.6261682242990654,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...",llama3_GSM8K_test,1,1,5,1,4


In [24]:
feature_li = ['QUA_IM', 'SIM_COT_BIGRAM', 'SIM_COT_AGG', 'SIM_AC_BIGRAM', 'SIM_AC_AGG', 'SIM_INPUT', 'STEP_COUNT', 'AVG_STEP_LENGTH', 'MATH_TERM_DENSITY', 'IMPERATIVE_DENSITY', 'STEP_COHERENCE'] 
df_llama_confidence_scores = trained_LR_model(df_llama_processed, feature_li, report_auroc=False) # Note that we keep the test data only to aviod overfit

Train DataFrame:
Model
llama3    35
Name: count, dtype: int64
Name
BigBench_easy            10
GSM8K_hard                8
MathQA_dev                5
BigBench_hard             4
GSM8K_test                4
MathQA_challenge_test     4
Name: count, dtype: int64

Test DataFrame:
Model
llama3    15
Name: count, dtype: int64
Name
BigBench_easy            4
GSM8K_hard               3
MathQA_challenge_test    2
MathQA_dev               2
GSM8K_test               2
BigBench_hard            2
Name: count, dtype: int64
Optimization terminated successfully.
         Current function value: 0.403831
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:            Correctness   No. Observations:                 1400
Model:                          Logit   Df Residuals:                     1388
Method:                           MLE   Df Model:                           11
Date:                Thu, 30 Jan 2025   Pseudo R-squ.:            

In [25]:
df_llama_confidence_scores.head()

,id,Name,Model,correct answer,CoT answers,Correctness,MATH_TERM_DENSITY,STEP_COHERENCE,SIM_AC_AGG,SIM_COT_AGG,...,DIF_IV,QUA_IM,STEP_COUNT,SIM_AC_BIGRAM,SC_correctness,ES_correctness,ES_steps,asc_correctness,asc_steps,confidence_score
0,40,BigBench_easy,llama3,C,"[D, C, A, E, E, C, E, A, D, A, C, E, E, D, D, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0.5, 0.37209302325581395, 0.45033112582781454...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...",0,0,40,0,40,"[0.23638707935418551, 0.10582059028380454, 0.1..."
1,7,MathQA_challenge_test,llama3,c,"[E, C, E, E, E, E, E, E, C, C, D, D, E, B, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.012269938650306749, 0.02030456852791878, 0....","[0, 0, 0, 0, 0.32393939393939397, 0, 0, 0.2148...","[0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, ...","[0.5, 0.3652173913043478, 0.38, 0.254545454545...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, ...","[0, 0, 0, 1, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 3, ...","[0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, ...",0,0,7,0,7,"[0.18175323341013105, 0.06319644962629163, 0.1..."
2,10,MathQA_dev,llama3,b,"[E, A, A, E, E, E, E, E, E, E, A, E, E, B, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.005050505050505051, 0.011764705882352941, 0...","[0.2361111111111111, 0.22580645161290322, 0, 0...","[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, ...","[0.5, 0.6931818181818181, 0.6274509803921569, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 0, ...","[0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, ...",0,0,8,0,10,"[0.18299950629311443, 0.41321052181956053, 0.4..."
3,44,MathQA_challenge_test,llama3,a,"[E, E, E, D, E, E, E, E, E, E, E, E, E, E, E, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.011764705882352941, 0.011494252873563218, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0.32205919503079744, ...","[0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0.5, 0.38497652582159625, 0.296137339055794, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,0,9,0,7,"[0.17491185964273384, 0.18835863334431088, 0.1..."
4,33,BigBench_easy,llama3,D,"[B, E, C, A, C, C, C, E, A, E, C, A, E, E, C, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0.5, 0.45112781954887216, 0.3549382716049383,...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...",0,0,40,0,40,"[0.24724127099954366, 0.18596011704801862, 0.0..."


In [26]:
N = 5
threshold = 0.5

# Applying early stopping mechanism
df_final = CS_early_stopping(df=df_llama_confidence_scores, threshold=threshold, N=N)

SC_ACC : 0.2
ES_ACC : 0.2
CS_ACC : 0.3333333333333333
SC_Avg_Steps : 40
ES_Avg_Steps : 21.6
CS_Avg_Steps : 34.266666666666666
ASC_Avg_Steps : 18.066666666666666
ASC_ACC : 0.2


# How to Run the Code the get CoTs?